### Tratamento das imagens em diferentes camadas

In [1]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    layers.Input(shape=(300,300,3)),

    layers.Resizing(300,300),
    layers.Rescaling(1./255),

    layers.Conv2D(32, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'),           
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(256, kernel_size=(3,3), activation='relu', strides=(1,1), padding='same'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),

    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),
    layers.Dropout(0.4),

    layers.Dense(64, activation=activations.relu, kernel_initializer=initializers.RandomNormal()),

    layers.Dense(1, activation=activations.sigmoid, kernel_initializer=initializers.RandomNormal())
])

### Configurando o optimizer

In [2]:
from tensorflow.keras import optimizers, losses, metrics

lr = 0.001

model.compile(
    optimizer = optimizers.Adam(
        learning_rate = lr    
    ),
    loss = losses.BinaryCrossentropy(),
    metrics = [ metrics.binary_accuracy ]
)

### Selecionando as imagens para o treinamento e teste

In [3]:
from tensorflow.keras import utils

path = './CatsVsDogs'
batch_size = 64

train = utils.image_dataset_from_directory(
    directory = path,
    shuffle = True,
    seed = 1,
    subset='training',
    validation_split = 0.3,
    image_size = (300,300),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    directory = path,
    shuffle = True,
    seed = 1,
    subset='validation',
    validation_split = 0.3,
    image_size = (300,300),
    batch_size = batch_size
)

Found 2000 files belonging to 2 classes.
Using 1400 files for training.
Found 2000 files belonging to 2 classes.
Using 600 files for validation.


### Realizando o treinamento do modelo

In [4]:
from tensorflow.keras import callbacks

model_path = "./catsvsdogs.keras"

patience = 5
epochs = 100

model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    verbose = True,
    
    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks .ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 57s 3s/step - binary_accuracy: 0.5014 - loss: 0.6981 - val_binary_accuracy: 0.5100 - val_loss: 0.6914
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 54s 2s/step - binary_accuracy: 0.5286 - loss: 0.6893 - val_binary_accuracy: 0.6217 - val_loss: 0.6720
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - binary_accuracy: 0.5693 - loss: 0.6740 - val_binary_accuracy: 0.5750 - val_loss: 0.6469
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - binary_accuracy: 0.6121 - loss: 0.6534 - val_binary_accuracy: 0.6483 - val_loss: 0.6254
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - binary_accuracy: 0.6207 - loss: 0.6378 - val_binary_accuracy: 0.6267 - val_loss: 0.6180
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - binary_accuracy: 0.6500 - loss: 0.6105 - val_binary_accuracy: 0.6550 - val_loss: 0.6079
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 55s 3s/step - binary_accuracy: 0.6457 - loss: 0.6309 - val_binary_accuracy: 0.6367 - val_loss: 0.6452
Epoch 8/100
2

: 